In [1]:
pip install gym gym[atari]

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym

In [3]:
env = gym.make('CartPole-v0')

In [8]:
env.reset()

array([-0.02049185, -0.04181021,  0.04333399,  0.04982264], dtype=float32)

In [7]:
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())